# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

В предыдущем задании вы собрали всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

Загрузите эту таблицу.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import joblib

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/nikitaely/linear_model_and_dev/main/df.csv')

Разбейте данные на тренировочную и тестовую часть в пропорции 80% к 20%, зафиксируйте `random_state = 42`.

In [3]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train

,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,PERSONAL_INCOME,AGREEMENT_RK,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,CLOSED_FL_x,CLOSED_FL_y
4459,35,1,2,2,1,0,18000.0,61081143,2,2,2,2
7475,30,1,0,0,1,0,10000.0,66973273,1,1,0,0
2182,37,1,2,2,1,0,10000.0,64471772,1,1,0,0
13086,31,1,2,2,1,0,12000.0,66890158,2,2,2,2
4115,32,1,0,0,1,0,11000.0,64353292,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5191,39,1,1,1,1,0,18000.0,65385165,1,1,1,1
13418,51,1,2,2,1,0,15000.0,74479701,1,1,1,1
5390,44,0,2,2,1,0,15000.0,62502792,1,1,1,1
860,56,1,2,0,1,0,30000.0,61372730,2,2,0,0


На тренировочных данных обучите линейную модель классификации для предсказания целевой переменной (столбец `TARGET`).

Сделайте прогноз вероятности отклика на рекламную кампанию для тестовых данных.

In [5]:
numeric = ['AGE', 'CHILD_TOTAL', 'DEPENDANTS', 'PERSONAL_INCOME', 'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED']
categorical = ['GENDER', 'DEPENDANTS', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL']

In [6]:
ct = ColumnTransformer([('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
                      ('scaler', MinMaxScaler(), numeric)])

pipe = Pipeline([('transformer', ct),
                ('LR_model', LogisticRegression())])

In [7]:
pipe.fit(X_train, y_train)

pred = pipe.predict_proba(X_test)

pred

C:\Users\nikel\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[0.97040231, 0.02959769],
       [0.89728326, 0.10271674],
       [0.83899536, 0.16100464],
       ...,
       [0.97794166, 0.02205834],
       [0.91502191, 0.08497809],
       [0.85780545, 0.14219455]])

Переведите вероятности в классы по стандартному порогу (0.5) и на тестовом наборе данных вычислите метрики:

* accuracy
* precision
* recall
* f1-score

In [8]:
new_pred = np.array(tuple(map(lambda x: int(x), pred[:, 1] > 0.5)))
print(f'accuracy: {accuracy_score(y_test, new_pred)}\nprecision: {precision_score(y_test, new_pred)}\nrecall {recall_score(y_test, new_pred)}\nf1: {f1_score(y_test, new_pred)}\n')

accuracy: 0.8850574712643678
precision: 1.0
recall 0.002849002849002849
f1: 0.005681818181818182



Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.

Но при этом точность не должна просесть, поэтому за ней тоже следим.

Разбейте тренировочные данные на `train` и `val` части в пропорции 3 к 1.

В цикле:

* переберите пороги от 0 до 1 с шагом 0.01
* вычислите для каждого порога значение метрик precision и recall
* подберите такой порог, при котором recall не меньше 0.66, а точность максимальна.

In [9]:
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

pipe.fit(X_train_new, y_train_new)
pred_val = pipe.predict_proba(X_val)

In [10]:
max_precision = 0
top_threshold = None
for threshold in np.arange(0, 1, 0.001):

  new_pred = np.array(list(map(lambda x: int(x), pred_val[:, 1] > threshold)))
  if recall_score(y_val, new_pred) >= 0.66 and precision_score(y_val, new_pred) > max_precision:
     max_precision = precision_score(y_val, new_pred)
     top_threshold = threshold

print(f'threshold:{top_threshold}')
new_pred = np.array(list(map(lambda x: int(x), pred_val[:, 1] > top_threshold)))
print(f'accuracy: {accuracy_score(y_val, new_pred)}\nprecision: {precision_score(y_val, new_pred)}\nrecall {recall_score(y_val, new_pred)}\nf1: {f1_score(y_val, new_pred)}\n')

threshold:0.121
accuracy: 0.5520525451559934
precision: 0.17259905977165882
recall 0.6606683804627249
f1: 0.2736954206602769



Для выбранного порога посчитайте все метрики на тестовых данных. Сильно ли они отличаются от метрик на валидации?

In [11]:
new_pred = np.array(list(map(lambda x: int(x), pred[:, 1] > top_threshold)))
print(f'accuracy: {accuracy_score(y_test, new_pred)}\nprecision: {precision_score(y_test, new_pred)}\nrecall {recall_score(y_test, new_pred)}\nf1: {f1_score(y_test, new_pred)}\n')

accuracy: 0.5257799671592776
precision: 0.14673561732385262
recall 0.6467236467236467
f1: 0.23919915700737618



Метрики на тесте немного просели, но сильных отличий от валидации нет.

Выведите на экран в виде таблицы топ-6 признаков с наибольшими по модулю весами модели.

In [12]:
pipe['LR_model'].coef_

array([[ 0.03007942, -0.03543552, -0.06977551, -0.24001723, -0.18603021,
         0.03080307, -0.18608212,  0.54452189,  0.10122403, -0.25923762,
         0.25388152,  0.18410278, -0.18945887, -1.1790615 ,  1.22724746,
         0.34432611,  3.46661188, -0.19442132, -0.19442132]])

Сохраним модель

In [15]:
joblib.dump(pipe, r'C:\Users\nikel\Projects\linear_model_and_dev_tools\model.pkl')

['C:\\Users\\nikel\\Projects\\linear_model_and_dev_tools\\model.pkl']

In [17]:
np.save(r'C:\Users\nikel\Projects\linear_model_and_dev_tools\pred', pred)
np.save(r'C:\Users\nikel\Projects\linear_model_and_dev_tools\y_test', y_test)

## Задание 2

Добавьте в Streamlit-приложение визуализацию результатов модели:

* опцию выбора порога и вывод метрик качества в зависимости от выбранного порога

* вывод прогноза модели на выбранном объекте (клиенте) - вероятность отклика на рекламу.

## Бонус

Попробуйте применить другие модели классификации для решения этой задачи (любые какие знаете).

Удалось ли добиться улучшения качества модели?

In [ ]:
# your code here